In [248]:
# imports
from imgaug import augmenters as iaa
import imgaug as ia
import os
import numpy as np
import cv2
import glob
from PIL import Image
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage


In [249]:
# seed for ia randomizer
ia.seed(1)


In [250]:
# fetch certain file types
images_names = []
ignored = 0

for fname in os.listdir('images'):
    if fname.endswith('.jpg'):
        images_names.append(fname)
    else:
        ignored += 1

print("found", len(images_names), ".jpg files")
print("ignored", ignored, "other file types")


found 9 .jpg files
ignored 9 other file types


In [251]:
# extract only bounding box information from .txt file (see format below)
def extract_bounding_boxes(file):
    bounding_boxes = []
    with open(file) as f:
        boxes = f.readlines()
        for box in boxes:
            box_arr = box.split(' ')
            bounding_boxes.append(
                [box_arr[1], box_arr[2], box_arr[3], box_arr[4].strip("\n")])
    return bounding_boxes

# extract only the classification label from .txt file (see format below)


def extract_classification_labels(file):
    class_labels = []
    with open(file) as f:
        labels = f.readlines()
        for label in labels:
            lab_arr = label.split(' ')
            class_labels.append(lab_arr[0])
    return class_labels


Bounding box .txt files should look as follows:

3 0.297656 0.615278 0.118750 0.205556   
3 0.622656 0.374306 0.056250 0.093056

Where the first number represents a classification label

In [252]:
# put images in np-array from memory & put corresponding bounding boxes in array in order
# bounding_box_images contains tuples in which the first element corresponds to the image and the second element corresponds to the bounding boxes details as an array
bounding_box_images = []
classification_labels = []

for image_name in images_names:
    # obtain image
    location = "images/"+image_name
    image = cv2.imread(location)

    # obtain bounding boxes
    loc = os.path.splitext(location)[0] + '.txt'
    bounding_boxes = extract_bounding_boxes(loc)

    # put data in tuple and append to array
    bounding_box_images.append((image, bounding_boxes))

    # save classification labels for future use
    classification_labels.append(extract_classification_labels(loc))


print("images found with bounding box(es):", len(bounding_box_images))
print("classifications found for boxes:", len(classification_labels))
print(classification_labels)


images found with bounding box(es): 9
classifications found for boxes: 9
[['3', '3'], ['3', '0'], ['0'], ['3'], ['0'], ['3', '3'], ['3', '3', '3', '3', '3'], ['0'], ['3']]


In [253]:
# sequential operations

seq = iaa.Sequential([
    iaa.Fliplr(1)
])


In [254]:



# only use when transforming images
def normalize_bounding_boxes(img_shape, bbs):
    width = img_shape[1]
    heigth = img_shape[0]
    normalized_bbs = []

    for bb in bbs:
        bb = [float(val) for val in bb]
        bb_normalized = [round(bb[0]/width, 6), round(bb[1]/heigth, 6),
                         round(bb[2]/width, 6), round(bb[3]/heigth, 6)]
        normalized_bbs.append(bb_normalized)
    return normalized_bbs

def denormalize_bounding_boxes(img_shape, bbs):
    width = img_shape[1]
    heigth = img_shape[0]
    denormalized_bbs = []
    
    for bb in bbs:
        bb = [float(val) for val in bb]
        bb_denormalized = [round(bb[0]*width, 6), round(bb[1]*heigth, 6),
                         round(bb[2]*width, 6), round(bb[3]*heigth, 6)]
        denormalized_bbs.append(bb_denormalized)
    return denormalized_bbs


In [255]:
# perform image & bounding box augmentation on all images
augmented_bb_images = []
for bb_img in bounding_box_images:
    
    # extract bounding boxes per image and create BoundingBox objects from data
    bb_img_denormalised = denormalize_bounding_boxes(bb_img[0].shape, bb_img[1])
    
    bbs = BoundingBoxesOnImage([
        BoundingBox(x1=float(bb[0]), y1=float(bb[1]), x2=float(bb[2]), y2=float(bb[3])) for bb in bb_img_denormalised
    ], shape=bb_img[0].shape)
    
    
    # Augment BBs and images.
    aug_img, aug_bbs = seq(image=bb_img[0], bounding_boxes=bbs) 
    print("augbbs:", aug_bbs)
    
    # normalize bounding boxes to yolo format
    aug_bbs = [[bb.x1, bb.y1, bb.x2, bb.y2] for bb in aug_bbs]
    
    aug_bbs = normalize_bounding_boxes(bb_img[0].shape, aug_bbs)
   
    
    aug_bb_img = [aug_img, aug_bbs]
    augmented_bb_images.append(aug_bb_img)
   


# add classification label to each box
for idx, images in enumerate(augmented_bb_images):
    print("new image with boxes...")
    for idx2, box in enumerate(images[1]):
        # print(box)
        label = classification_labels[idx][idx2]
        
        label_box = [label]
        for b in box:
            label_box.append(b)

        augmented_bb_images[idx][1][idx2] = label_box
        print(augmented_bb_images[idx][1])
        
        

#TODO: IMPLEMENTEER:
# https://stackoverflow.com/questions/56115874/how-to-convert-bounding-box-x1-y1-x2-y2-to-yolo-style-x-y-w-h

augbbs: BoundingBoxesOnImage([BoundingBox(x1=899.0003, y1=148.0003, x2=1128.0000, y2=443.0002, label=None), BoundingBox(x1=483.0003, y1=67.0003, x2=1208.0000, y2=269.5003, label=None)], shape=(720, 1280, 3))
augbbs: BoundingBoxesOnImage([BoundingBox(x1=200.9997, y1=104.9998, x2=1176.0000, y2=477.4997, label=None), BoundingBox(x1=445.9994, y1=70.9999, x2=981.9994, y2=139.9997, label=None)], shape=(720, 1280, 3))
augbbs: BoundingBoxesOnImage([BoundingBox(x1=635.0003, y1=360.4997, x2=956.5005, y2=718.9999, label=None)], shape=(720, 1280, 3))
augbbs: BoundingBoxesOnImage([BoundingBox(x1=1062.4998, y1=49.0003, x2=1219.0003, y2=554.5001, label=None)], shape=(720, 1280, 3))
augbbs: BoundingBoxesOnImage([BoundingBox(x1=540.0000, y1=385.9999, x2=775.0003, y2=660.0002, label=None)], shape=(720, 1280, 3))
augbbs: BoundingBoxesOnImage([BoundingBox(x1=672.9997, y1=69.9998, x2=1172.0000, y2=355.9997, label=None), BoundingBox(x1=269.5002, y1=140.9998, x2=968.9997, y2=649.4998, label=None)], shape=(72

In [256]:
# augmented_bb_images folder path
path = 'augmented_bb_images/'


In [257]:
# ONLY EXECUTE THIS CELL BLOCK IF YOU WANT THE CURRENT AUGMENTED IMAGES FOLDER TO BE DELETED!!!

# empty augmented_images folder

for f in os.listdir(path):
    os.remove(os.path.join(path, f))


In [258]:
for idx, images in enumerate(augmented_bb_images):

    # save augmented image
  
    name =  "AUG-" + images_names[idx] 
    
    cv2.imwrite(os.path.join(path, name), images[0]) 

    # save bounding boxes in yolo format to .txt file
    boxes = images[1]
    name =  path + "AUG-" + os.path.splitext(images_names[idx])[0] + '.txt'

    f = open(name, 'w+')
    print("writing to:", name)
    for idx2, box in enumerate(boxes):
        # print(augmented_bb_images[idx][1])
        box_string = ""
        for item in box:
            box_string += str(item) + " "
        box_string += "\n"
        try:
            f.write(box_string)
        except Exception as ex:
            print("something went wrong when saving bounding boxes for image:", name)
            print(ex)

    f.close()
    print("stopped writing to:", name)


writing to: augmented_bb_images/AUG-0ab0016e-989c-4978-ac70-cca24e094eb3.txt
stopped writing to: augmented_bb_images/AUG-0ab0016e-989c-4978-ac70-cca24e094eb3.txt
writing to: augmented_bb_images/AUG-0c3f985a-9024-4db5-a30d-05f15f35627a.txt
stopped writing to: augmented_bb_images/AUG-0c3f985a-9024-4db5-a30d-05f15f35627a.txt
writing to: augmented_bb_images/AUG-0e6c765d-8be2-4fda-93af-c3bdced6a9a2.txt
stopped writing to: augmented_bb_images/AUG-0e6c765d-8be2-4fda-93af-c3bdced6a9a2.txt
writing to: augmented_bb_images/AUG-0e495109-b241-4db2-bfb8-4bf6e38aa647.txt
stopped writing to: augmented_bb_images/AUG-0e495109-b241-4db2-bfb8-4bf6e38aa647.txt
writing to: augmented_bb_images/AUG-0eab23ec-fedb-4851-bafe-eab493059a8e.txt
stopped writing to: augmented_bb_images/AUG-0eab23ec-fedb-4851-bafe-eab493059a8e.txt
writing to: augmented_bb_images/AUG-1aa958e9-e861-4357-b1bd-001855ec2ec2.txt
stopped writing to: augmented_bb_images/AUG-1aa958e9-e861-4357-b1bd-001855ec2ec2.txt
writing to: augmented_bb_ima

TODO:   
De input van seq() verwacht pixeldata voor bounding boxes, nu is het nog scalings van bb's
dit moet dus worden aangepast om de bb's goed te transformeren.
